In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, cnn_clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.7/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
lyrics, max_len, word_list = cnn_clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
words = []
words.append('<PAD>')
for word in word_list:
    words.append(word)

In [8]:
train_lyrics = np.zeros(shape=(len(train_df), max_len))

row_index = 0
for row in train_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        train_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [9]:
print(train_lyrics.shape)

(10098, 1525)


In [10]:
train_genres = one_hot_genres(train_df['Genre'])

In [11]:
val_df = pd.read_csv("validation.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(val_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    val_df['Lyrics'][index] = text
    index+=1
del lyrics

In [12]:
val_genres = one_hot_genres(val_df['Genre'])

In [13]:
val_lyrics = np.zeros(shape=(len(val_df), max_len))

row_index = 0
for row in val_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list and col_index < max_len:
            val_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [14]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_len, dummy_list = cnn_clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [15]:
test_genres = one_hot_genres(test_df['Genre'])

In [16]:
test_lyrics = np.zeros(shape=(len(test_df), max_len))

row_index = 0
for row in test_df['Lyrics']:
    single_words = row.split()
    col_index = 0
    for word in single_words:
        if word in word_list and col_index < max_len:
            test_lyrics[row_index][col_index] = words.index(word)
        col_index+=1
    row_index+=1

In [17]:
print(train_lyrics.shape)

(10098, 1525)


In [18]:
# Source: https://github.com/ibrahimcelenli/cnn-word2vec-tweets-classification/blob/master/cnn_word2vec_tweets_classification.ipynb

word2vec = {}
with open('w2v_sg_model.word2vec', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:31], dtype='float32')
        word2vec[word] = vec

In [19]:
# Source: https://github.com/ibrahimcelenli/cnn-word2vec-tweets-classification/blob/master/cnn_word2vec_tweets_classification.ipynb

EMBEDDING_DIM = 30
embedding_matrix = np.zeros((len(words), EMBEDDING_DIM))
index = 0
for word in words:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
    index+=1


In [20]:
print(embedding_matrix.shape)

(34261, 30)


In [21]:
# For generating loss curve *Add callback function to model fitting*
%load_ext tensorboard

In [22]:
import tensorflow as tf
import datetime
!rm -rf ./logs/ 

In [23]:
log_dir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)


In [24]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 11859), started 1 day, 6:38:02 ago. (Use '!kill 11859' to kill it.)

In [196]:
from keras.layers import Embedding

embedding_layer = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=True)

embedding_layer2 = Embedding(len(words),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=True)

In [197]:
from keras.models import Model
from keras.layers import Dense, Input, Activation, Conv1D, MaxPooling1D, Dropout, Flatten
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model

sequence_input = Input(shape=(max_len,), dtype='int32')
embedding = embedding_layer(sequence_input)

# channel 1
conv1 = Conv1D(64, 4, activation='relu')(embedding)
pool1 = MaxPooling1D(5)(conv1)
drop1 = Dropout(0.5)(pool1)
flat1 = Flatten()(drop1)

# channel 2
sequence_input2 = Input(shape=(max_len,), dtype='int32')
embedded2 = embedding_layer2(sequence_input2)
conv2 = Conv1D(64, 5, activation='relu')(embedded2)
pool2 = MaxPooling1D(5)(conv2)
drop2 = Dropout(0.5)(pool2)
flat2 = Flatten()(drop2)

# merge
merged = concatenate([flat1, flat2], axis=-1)
drop3 = Dropout(0.5)(merged)
# interpretation
dense = Dense(128, activation='relu')(drop3)
outputs = Dense(8, activation='softmax')(dense)


model = Model(inputs=[sequence_input, sequence_input2], outputs=outputs)


plot_model(model, show_shapes=True, to_file='cnn.png')
model.summary()

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_59 (InputLayer)           (None, 1525)         0                                            
__________________________________________________________________________________________________
input_60 (InputLayer)           (None, 1525)         0                                            
__________________________________________________________________________________________________
embedding_59 (Embedding)        (None, 1525, 30)     1027830     input_59[0][0]                   
__________________________________________________________________________________________________
embedding_60 (Embedding)        (None, 1525, 30)     1027830     input_60[0][0]                   
___________________________________________________________________________________________

In [198]:
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc'])

model.fit([train_lyrics,train_lyrics], train_genres, epochs=4, 
          shuffle=True, batch_size=16, validation_data=([val_lyrics,val_lyrics], val_genres),
          callbacks=[tensorboard_callback])

Train on 10098 samples, validate on 2142 samples
Epoch 1/4
10098/10098 [==============================] - 42s 4ms/step - loss: 1.6822 - acc: 0.3576 - val_loss: 1.5545 - val_acc: 0.4174
Epoch 2/4
10098/10098 [==============================] - 44s 4ms/step - loss: 1.4347 - acc: 0.4568 - val_loss: 1.4727 - val_acc: 0.4500
Epoch 3/4
10098/10098 [==============================] - 43s 4ms/step - loss: 1.2738 - acc: 0.5249 - val_loss: 1.5405 - val_acc: 0.4290
Epoch 4/4
10098/10098 [==============================] - 45s 4ms/step - loss: 1.1231 - acc: 0.5839 - val_loss: 1.4666 - val_acc: 0.4837


In [199]:
model.fit([val_lyrics,val_lyrics], val_genres, epochs=4, 
          shuffle=True, batch_size=16)

Epoch 1/4
2142/2142 [==============================] - 9s 4ms/step - loss: 1.5632 - acc: 0.4234
Epoch 2/4
2142/2142 [==============================] - 9s 4ms/step - loss: 1.2829 - acc: 0.5275
Epoch 3/4
2142/2142 [==============================] - 9s 4ms/step - loss: 1.0925 - acc: 0.5878
Epoch 4/4
2142/2142 [==============================] - 9s 4ms/step - loss: 0.8614 - acc: 0.6923


In [200]:
results = model.evaluate([test_lyrics,test_lyrics], test_genres, batch_size=16)
print("Accuracy: ", results[1])

2160/2160 [==============================] - 1s 599us/step
Accuracy:  0.4763889014720917
